In [8]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("DSPy")

<Experiment: artifact_location='mlflow-artifacts:/374362034103955121', creation_time=1741686562632, experiment_id='374362034103955121', last_update_time=1741686562632, lifecycle_stage='active', name='DSPy', tags={}>

In [9]:
mlflow.dspy.autolog()

In [1]:
import dspy

gpt4o_mini = dspy.LM('openai/gpt-4o-mini', max_tokens=2000)
gpt4o = dspy.LM('openai/gpt-4o', max_tokens=2000)
dspy.configure(lm=gpt4o_mini)  # we'll use gpt-4o-mini as the default LM, unless otherwise specified

/Users/aidand/dev/hello-dspy/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dspy.datasets import MATH

dataset = MATH(subset='algebra')
print(len(dataset.train), len(dataset.dev))

Generating test split: 100%|██████████| 1187/1187 [00:00<00:00, 414024.02 examples/s]

350 350


In [3]:
example = dataset.train[0]
print("Question:", example.question)
print("Answer:", example.answer)

Question: The doctor has told Cal O'Ree that during his ten weeks of working out at the gym, he can expect each week's weight loss to be $1\%$ of his weight at the end of the previous week. His weight at the beginning of the workouts is $244$ pounds. How many pounds does he expect to weigh at the end of the ten weeks? Express your answer to the nearest whole number.
Answer: 221


In [10]:
module = dspy.ChainOfThought("question -> answer")
module(question=example.question)

Prediction(
    reasoning="Cal O'Ree's weight loss each week is $1\\%$ of his weight at the end of the previous week. This means that at the end of each week, he will weigh $99\\%$ of his weight from the previous week. \n\nWe can express this mathematically. If his initial weight is $W_0 = 244$ pounds, then his weight at the end of week $n$ can be calculated using the formula:\n\n\\[\nW_n = W_{n-1} \\times 0.99\n\\]\n\nThis can be simplified to:\n\n\\[\nW_n = W_0 \\times (0.99)^n\n\\]\n\nFor $n = 10$ weeks, we have:\n\n\\[\nW_{10} = 244 \\times (0.99)^{10}\n\\]\n\nNow we calculate $(0.99)^{10}$:\n\n\\[\n(0.99)^{10} \\approx 0.904382\n\\]\n\nNow we can calculate Cal O'Ree's weight after 10 weeks:\n\n\\[\nW_{10} \\approx 244 \\times 0.904382 \\approx 220.5\n\\]\n\nRounding to the nearest whole number, Cal O'Ree can expect to weigh approximately $221$ pounds at the end of the ten weeks.",
    answer='221'
)

Trace(request_id=fe47f1f8059e4ee7888525bf0322b6a2)

In [7]:
dspy.inspect_history(n=1)





[2025-03-13T12:44:05.583790]

System message:

Your input fields are:
1. `question` (str)

Your output fields are:
1. `reasoning` (str)
2. `answer` (str)

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `question`, produce the fields `answer`.


User message:

[[ ## question ## ]]
The doctor has told Cal O'Ree that during his ten weeks of working out at the gym, he can expect each week's weight loss to be $1\%$ of his weight at the end of the previous week. His weight at the beginning of the workouts is $244$ pounds. How many pounds does he expect to weigh at the end of the ten weeks? Express your answer to the nearest whole number.

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## a

In [11]:
THREADS = 24
kwargs = dict(num_threads=THREADS, display_progress=True, display_table=5)
evaluate = dspy.Evaluate(devset=dataset.dev, metric=dataset.metric, **kwargs)

evaluate(module)

  0%|          | 0/350 [00:00<?, ?it/s]

Average Metric: 261.00 / 350 (74.6%): 100%|██████████| 350/350 [02:12<00:00,  2.64it/s]

2025/03/13 12:47:12 INFO dspy.evaluate.evaluate: Average Metric: 261 / 350 (74.6%)


,question,example_reasoning,example_answer,pred_reasoning,pred_answer,method
0,What is the smallest integer value of $c$ such that the function $...,The given function has a domain of all real numbers if and only if...,1,To determine the smallest integer value of \( c \) such that the f...,1,✔️ [True]
1,What is the least value of $x$ that is a solution of $|{-x+3}|=7$?,"In order to have $|{-x+3}| = 7$, we must have $-x + 3 = 7$ or $-x ...",-4,"To solve the equation \( |{-x+3}|=7 \), we need to consider the de...",-4,✔️ [True]
2,Evaluate $\left\lceil -\frac{7}{4}\right\rceil$.,"$-\frac{7}{4}$ is between $-1$ and $-2$, so $\left\lceil -\frac{7}...",-1,"To evaluate \(\left\lceil -\frac{7}{4}\right\rceil\), we first nee...",-1,✔️ [True]
3,"A triangle has vertices at coordinates $(11,1)$, $(2,3)$ and $(3,7...",We must find the distance between each pair of points by using the...,10,To find the length of the longest side of the triangle with vertic...,10,✔️ [True]
4,Let $f(x) = x + 2$ and $g(x) = 1/f(x)$. What is $g(f(-3))$?,"First, we find that $f(-3) = (-3) + 2 = -1$. Then, $$g(f(-3)) = g(...",1,"To find \( g(f(-3)) \), we first need to evaluate \( f(-3) \). The...",1,✔️ [True]


74.57

[Trace(request_id=280fe9fba2124f67a5144faaee55ca90), Trace(request_id=ed5ca26a028841a984314a79aaec3da6), Trace(request_id=ce9fc3f4520449ebad29a4aa85df76ae), Trace(request_id=0f55e531d255460ba9c6b27f5fae16be), Trace(request_id=7d2e458993174dca969782d086d75a69), Trace(request_id=9acdc44b72b644a398756af3860660c8), Trace(request_id=460e9381c47d4de9b796cc6dc2d781e3), Trace(request_id=423cdf11401540b886ebdebf59dc1b55), Trace(request_id=94b9795590cb42ebbc02ac3b9b27c97d), Trace(request_id=83aab480f2f142c3ba5e4a00a0df5386)]

In [ ]:
kwargs = dict(num_threads=THREADS, teacher_settings=dict(lm=gpt4o), prompt_model=gpt4o_mini)
optimizer = dspy.MIPROv2(metric=dataset.metric, auto="medium", **kwargs)

kwargs = dict(requires_permission_to_run=False, max_bootstrapped_demos=4, max_labeled_demos=4)
optimized_module = optimizer.compile(module, trainset=dataset.train, **kwargs)

2025/03/13 12:49:28 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: True
num_candidates: 19
valset size: 280

2025/03/13 12:49:28 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/03/13 12:49:28 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/03/13 12:49:28 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=19 sets of demonstrations...


Bootstrapping set 1/19
Bootstrapping set 2/19
Bootstrapping set 3/19


  0%|          | 0/70 [00:00<?, ?it/s]